In [1]:
# Import libraries

import sys
import logging
import pandas as pd
import xarray as xr
from pathlib import Path

root_path = Path(globals()['_dh'][0]).resolve().parent
sys.path.append(str(root_path))

import paths
from library.assumptions import read_assumptions
from library.weather import load_weather
from library.network import build_network

logging.basicConfig(level=logging.INFO)

In [2]:
# Set the configuration

## Parameters you won't change very often
base_currency = 'SEK'
exchange_rates = {
    'EUR': 11.68,
    'USD': 10.70
}
base_year = 2024
discount_rate = 0.05
onwind_turbine =  "2030_5MW_onshore.yaml"
offwind_turbine = "2030_20MW_offshore.yaml"
resolution = 3

## Parameters that will change frequently
target_year = 2030
use_offwind = True
use_h2 = True
h2_initial = 1000
biogas_limit = 500
load_target = 15 #TWh

In [3]:
# Load the data needed from assumptions, the electricity demand, and the atlite output from ERA5 weather data for VGR 2023

## Transform assumptions to range base_year to target_year
assumptions = read_assumptions(paths.input_path / 'assumptions.csv', base_year, target_year, base_currency, exchange_rates, discount_rate)

# Read the normalized demand from csv file (see normalize_demand() in library.demand for details)
# And then calculate target_load using projection of energy need in target_year
normalized_demand = pd.read_csv(paths.input_path / 'demand/normalized-demand-2023-3h.csv', delimiter=',')
target_load = load_target * normalized_demand['value'].values.flatten() * 1_000_000

# Create of load the cutout from atlite (we assume weather data from 2023 and a 3h window)
geo = '14' # All of VGR
section = None
cutout, selection, index = load_weather(geo, section, '2023-01', '2023-12')
geography = selection.total_bounds  

capacity_factor_solar = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-solar.nc").values.flatten()
capacity_factor_onwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-onwind.nc").values.flatten()
capacity_factor_offwind = xr.open_dataarray(paths.input_path / 'renewables' / f"capacity-factor-{geo}-2023-01-2023-12-offwind.nc").values.flatten()

In [4]:
# Build the network

network = build_network(index, resolution, geography, target_load, assumptions, capacity_factor_solar, capacity_factor_onwind, capacity_factor_offwind, use_offwind, use_h2, h2_initial, biogas_limit)

In [5]:
# Add constraints to the model and run the optimization

## Create the model
model = network.optimize.create_model()

## Add offwind constraint
if False:
    generator_capacity = model.variables["Generator-p_nom"]
    offwind_percentage = 0.5

    offwind_constraint = (1 - offwind_percentage) / offwind_percentage * generator_capacity.loc['offwind'] - generator_capacity.loc['onwind']
    model.add_constraints(offwind_constraint == 0, name="Offwind_constraint")

## Add battery charge/discharge ratio constraint
link_capacity = model.variables["Link-p_nom"]
lhs = link_capacity.loc["battery-charge"] - network.links.at["battery-charge", "efficiency"] * link_capacity.loc["battery-discharge"]
model.add_constraints(lhs == 0, name="Link-battery_fix_ratio")

## Run optimization
network.optimize.solve_model(solver_name='highs')

INFO:linopy.model: Solve problem using Highs solver
INFO:linopy.io:Writing objective.
Writing integer variables.: 100%|██████████| 2/2 [00:00<00:00, 672.81it/s]
INFO:linopy.io: Writing time: 0.53s
INFO:linopy.solvers:Log file at /tmp/highs.log


Running HiGHS 1.7.2 (git hash: 184e327): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [4e-06, 3e+02]
  Cost   [8e+01, 2e+06]
  Bound  [0e+00, 0e+00]
  RHS    [5e+02, 3e+03]
Presolving model
48184 rows, 42352 cols, 127032 nonzeros  0s
38071 rows, 32239 cols, 117366 nonzeros  0s
37991 rows, 32159 cols, 117722 nonzeros  0s

Solving MIP model with:
   37991 rows
   32159 cols (0 binary, 5 integer, 0 implied int., 32154 continuous)
   117722 nonzeros

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
     Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Confl. | LpIters     Time

         0       0         0   0.00%   -inf            inf                  inf        0      0      0         0     0.2s
 S       0       0         0   0.00%   -inf            22840527146.16     Large        0      0      0         0     8.7s
 R       0       0         

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 46739 primals, 105141 duals
Objective: 2.28e+10
Solver model: available
Solver message: optimal

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, Store-energy_balance were not assigned to the network.


('ok', 'optimal')

In [6]:
network.statistics()

Optimal Capacity  Installed Capacity        Supply  \
Generator backstop       2245.061126                 0.0  4.923503e+05   
          biogas          500.000000                 0.0  6.551018e+05   
          offwind           0.000000                 0.0  5.450919e-14   
          onwind         6009.197143                 0.0  7.826806e+06   
          solar          6438.919620                 0.0  6.656816e+06   
Link      AC             4501.856142                 0.0  1.418579e+07   
          biogas          500.000000                 0.0  6.551018e+05   
          h2              700.000000                 0.0  2.094201e+05   
          li-ion         1914.697792                 0.0  2.523178e+06   
          mixedgas        600.000000                 0.0  4.361320e+05   
Load      -                 0.000000                 0.0  0.000000e+00   
Store     h2             3169.348482                 0.0  1.047101e+05   
          li-ion        18453.631778                 0.0  2.596529e+06   

                      Withdrawal      Dispatch  Transmission  Capacity Factor  \
Generator backstop  0.000000e+00  4.923503e+05  0.000000e+00         0.025035   
          biogas    0.000000e+00  6.551018e+05  0.000000e+00         0.149567   
          offwind   0.000000e+00  5.450919e-14  0.000000e+00              NaN   
          onwind    0.000000e+00  7.826806e+06  0.000000e+00         0.148684   
          solar     0.000000e+00  6.656816e+06  0.000000e+00         0.118018   
Link      AC        1.429807e+07 -1.122824e+05  1.154834e+07         0.362561   
          biogas    6.551018e+05  0.000000e+00  6.551018e+05         0.149567   
          h2        2.902628e+05 -8.084268e+04  0.000000e+00         0.047336   
          li-ion    2.630596e+06 -1.074177e+05  0.000000e+00         0.156837   
          mixedgas  7.598119e+05 -3.236799e+05  7.598119e+05         0.144561   
Load      -         1.500000e+07 -1.500000e+07  0.000000e+00              NaN   
Store     h2        1.047101e+05  2.751221e-11  0.000000e+00         0.358298   
          li-ion    2.603380e+06 -6.850995e+03  0.000000e+00         0.529801   

                     Curtailment  Capital Expenditure  \
Generator backstop  0.000000e+00         0.000000e+00   
          biogas    0.000000e+00         0.000000e+00   
          offwind   0.000000e+00         0.000000e+00   
          onwind    1.232089e+07         8.100161e+09   
          solar     6.170481e+05         3.344542e+09   
Link      AC        0.000000e+00         3.918955e+08   
          biogas    0.000000e+00         0.000000e+00   
          h2        0.000000e+00         1.091117e+08   
          li-ion    0.000000e+00         0.000000e+00   
          mixedgas  0.000000e+00         7.989853e+08   
Load      -         0.000000e+00         0.000000e+00   
Store     h2        0.000000e+00         2.416216e+08   
          li-ion    0.000000e+00         3.554022e+09   

                    Operational Expenditure  Revenue  Market Value  
Generator backstop             5.175587e+09      0.0           NaN  
          biogas               8.370499e+08      0.0           NaN  
          offwind              3.152538e-12      0.0           NaN  
          onwind               2.252522e+08      0.0           NaN  
          solar                0.000000e+00      0.0           NaN  
Link      AC                   0.000000e+00      0.0           NaN  
          biogas               0.000000e+00      0.0           NaN  
          h2                   0.000000e+00      0.0           NaN  
          li-ion               0.000000e+00      0.0           NaN  
          mixedgas             4.889808e+07      0.0           NaN  
Load      -                    0.000000e+00      0.0           0.0  
Store     h2                   0.000000e+00      0.0           NaN  
          li-ion              -1.921944e+05      0.0           NaN